# INTRODUCTION TO JULIA

## Warning

+ Setting: Phd seminar in Prague 2011. MK talks about _FEniCS_. Wrongly, people fall asleep. __FEniCS is awesome__.
+ Setting: Phd seminar in Oslo 2016. MK talks about _Julia_. ???. __Julia is awesome__.

## SETUP

+ Ubuntu package from the repository
```
sudo add-apt-repository ppa:staticfloat/juliareleases
sudo add-apt-repository ppa:staticfloat/julia-deps
sudo apt-get update
sudo apt-get install julia
```
+ [Download](http://julialang.org/downloads/) Binaries (all platforms)
+ Build from source code
```
git clone https://github.com/JuliaLang/julia.git
cd julia
git checkout release-0.4  # unless you want the latest master
make -j 4
```
+ Try Julia in your browser via [JuliaBox](https://www.juliabox.org/). No installation but Google account required


## INTERACTION & SNEAK PEEK

Some modes of interacting with Julia:
+ REPL (go to terminal)
    - Julian mode, help mode(?), shell mode(;)
    - ! `edit`
+ text editor + command line
    - ! Demo
+ Jupyter notebook
    - From inside Julia run `Pkg.add("IJulia")`. This gives you Julia kernel that the notebook can talk to
    - This is the notebok :)

In [ ]:
installed_packages = Pkg.installed()
# len tab
length(installed_packages)

In [ ]:
haskey(installed_packages, "IJulia")

In [ ]:
# Some unicode
a = 1:10;
1 ∈ a

In [ ]:
b = 4:20;
a ∩ b

In [ ]:
# Explore: Docstring contains the query
apropos("eigenvalues")

## WHAT IS IT?
+ Julia: A fresh approach to numerical computing, see [paper](http://arxiv.org/abs/1411.1607)
+ High level, dynamic, (dynamically) typed, general purpose language
+ Free and open-source. MIT licence
+ Went [viral](https://www.google.com/trends/explore#q=julia%20language) on Valentine's day 2014
+ Some features we will see in action today
    - rich typestem
    - type inference
    - JIT compiler
    - multiple dispatch
    - metaprogramming
    - built in parallelism

## WHY DO WE NEED YET ANOTHER LANGUAGE?

Short answer - to address the two language paradygm
+ Express the idea in high level language(Python) - setup prototype quickly. Performance far from production code
+ Production code then written in low level language (C/C++)
+ Write performance critical parts of code in low level language and glue together (Numba, SWIG)
    - FEniCS uses Python to generate C++ code that is compiled and wrapped for Python
    - Diako/Mikael prototype in Python/NumPy and then write the same code in Cython
+ To extend NumPy/SciPy you (usually) have to write C

Julia's approach
+ Designed to be easy to write with performance close to C
+ If that is true you need one language to solve your problem
+ It seems to be true

### Some benchmark problems of our own

### Project Euler problem 14. 
The following iterative sequence is defined for the set of positive integers:

+ n → n/2 (n is even)
+ n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence: 13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1. Which starting number, under one million, produces the longest chain? Note that whether the chain terminates for arbitrary $n$ is an unproved conjecture.

+ To solve PE problem's the solution must be obtained in less then 60s. Honor code is to beat 1s.
+ Note that the code presented here is not optimal. It is rather a brute force solution.
+ The answer it produces is correct

In [ ]:
"""Compute the Collatz chain for number n."""
function collatz_chain(n)
    k = 1
    while n > 1
        n = isodd(n) ? 3n+1 : n >> 1
        k += 1
        # println(n)
    end
    k
end

"""Which of the number [1, stop) has the longest Collatz chain."""
function solve_euler(stop)
    n, N, N_max = 1, 0, 0  
    while n < stop
        value = collatz_chain(n)
        if value > N_max
            N = n
            N_max = value
        end
        n += 1
    end
    (N, N_max)
end

In [ ]:
# Let's time it
N = 1000000
t0 = tic()
answer = solve_euler(N)
t1 = toc();
answer, t1

Julia is done in about 0.3s, Python need 12s. So with Julia I am onto the next problem. With Python it is back to the drawing board.

### Julia fractal
+ It is only approprate that we explore the Julia set
+ Explore points $z$ such that $z_{n+1}=z^2_{n}+c$ is bounded for any n

In [ ]:
# Julia code is a translation of the definition
"""Color Julia set. Code from https://www.youtube.com/watch?v=PsjANO10KgM"""
function julia(z, c)
    for n in 1:80
        if abs2(z) > 4
            return n-1
        end
        z = z*z + c
    end
    return 80
end

In [ ]:
# We explore few points
cs = (complex(-0.06, 0.67), complex(0.279, 0), complex(-0.4, 0.6), complex(0.285, 0.01))

x = collect(1:-0.002:-1);
y = collect(-1.5:0.002:1.5);

Js = []
# Evaluate fractal generation
t0 = tic()
for c in cs
    #push!(Js, [julia(complex(r, i), c) for i in x, r in y]);
    push!(julia_faster(x, y, c))
end
t1 = toc()

@printf("Generated in %.4f s\n", t1);
println("Image size $(size(J, 1))x$(size(J, 2))")

In [ ]:
# Here we use python interopt(more on that later) to plot the fractals
using PyCall
@pyimport matplotlib.pyplot as plt

for J in Js
    plt.figure()
    plt.imshow(J, cmap="viridis", extent=[-1.5, 1.5, -1, 1])
end
plt.show()

About 4 seconds is required to make the fractals. This is slower then Python which takes about 2.8. However the speed comes from clever vectorization. And don't worry the code above can be made faster.

## WHEN IT READS LIKE PYTHON

+ Illustrate (not unrealistic) workflow by a made of postprocessing example
+ Note that this is high-level: there are lambdas `x -> x + 1`, higher order functions, containers
+ No type declarations

In [ ]:
# Read the content on the directory and keep only the files
files = filter(isfile, readdir("."))

In [ ]:
# How many of the files are Julia files
n_jl_files = count(f -> endswith(f, ".jl"), files)
println("$(pwd()) contains $(n_jl_files) files.")

In [ ]:
# Get those files and print them
jl_files = files[find(f -> endswith(f, ".jl"), files)]
for (i, file) in enumerate(jl_files)
    println("Julia file $(file)")
end

In [ ]:
# We have seen Array container. Let's show Sets
jl_files2 = append!(jl_files, jl_files)
jl_set = Set(jl_files2)
println("$(pwd()) contains $(length(jl_set)) files.")

In [ ]:
# Reading from file. How many words in the MIT licence?
LICENCE = open("LICENSE")
content = lowercase(readall(LICENCE))
words = split(content, r"\W", keep=false)
close(LICENCE)
length(words)

In [ ]:
# Count them with multiplicity
unique = Dict()
for word in words
    unique[word] = get(unique, word, 0) + 1
end

In [ ]:
# Most common ones
for (k, v) in sort(collect(unique), by=last, rev=true)
    println("$(k) $(v)")
end

## SOME DIFFERENCES FROM PYTHON

A few examples where the behaviour from Python is different

In [ ]:
# : is => in the dictionary construction
dict = Dict(1 => "Miro", 2 => "Kuchta")

In [ ]:
# in checks for (key, value) pair
haskey(dict, 1), (1 => "Miro") ∈ dict

In [ ]:
# iteration is over (key, values) pairs
for (key, value) in dict
    println("$key => $value")
end

In [ ]:
# Sting concatenation is done with *. With + the operation should be commutative, which is not the case here
"Julia " * " Language"

In [ ]:
# / is for floating point division. Integer division is done by div
/(7, 4), div(7, 4)

In [ ]:
# Indexing starts at one. The last element in range is included
a = [1, 2, 3]
try
    a[0]
catch BoundsError
    println("No no, a[1]=", a[1], " ", a[end] == a[3] == 3)
end

collect(1:10)[10] == 10

In [ ]:
# complex numbers
complex(1, 1) == 1 + 1im

In [ ]:
# Convenience
x = 4
y = 2x  # no *
y == 8

In [ ]:
# More convenience. Save one "for"
[(i, j) for i in 1:4, j in "abcd"]

In [ ]:
# The arrays are column majored. In particular vector is a column vector
a = [1, 2, 3]

In [ ]:
# Contrast this with Python
A = reshape(collect(1:25), (5, 5))

In [ ]:
# In numpy you get to iterate over rows of matrix (for vectorization). In Julia this is not default. 
entries = [e for e in A]  #join(map(string, collect(A)), ", ")
# Moverover, columns are contiguous so accessing rows is not efficient.
# Finaly you iterate over values not pointers
for a in A
    a += 1
end
A

In [ ]:
# Column majored linear indexing
A[8] == 8

Now onto functions. Recall that you have already seen lambas `x -> 2x`

In [ ]:
# Multiline lambdas
map(f -> begin
    name, ext = splitext(f)
    uppercase(name)*uppercase(ext)^2
    end, readdir("."))

In [ ]:
# One lined named function definition
f(x, y) = 2x + 4y
f(1, 0) == 2

In [ ]:
# Implicit return statements
function f(name)
    if isequal(name, "Miro")
        return "slav"
    end
    "!"
end

(f("Miro"), f("You")) == ("slav", "!")

In [ ]:
# Variable positional arguments
mysum(args...) = join(map(string, args), "+")
mysum(10, 20)

In [ ]:
# Keyword arguments. Not a dictionary
function foo(args...; kwargs...)
    kwargs = Dict([k => v for (k, v) in kwargs])
    ans = join(map(string, args), "?")
    if haskey(kwargs, :x)  # Ignore :x for a while. It comes from the 'strange' kwargs array
        ans = ans * "?" * string(kwargs[:x])
    end
    ans 
end
foo(10, 20, x=40)

## WHEN IT _IS_ PYTHON

+ Not entirely sure about giving up Python? What if you could use Julia with Python to make the transition easier?
+ The whole Python ecosystem can be reached via Pkg __PyCall__

In [ ]:
using PyCall

In [ ]:
# Let's load numpy and have it compute for us the eigenvalues of some matrix
@pyimport numpy.linalg as npla
A = diagm([1, 2, 3, 4])          # Julia matrix
eigv, eigw = npla.eig(A)         # Passed to python no copying
eigv                             # The result is Julia Array

In [ ]:
# Let's have numpy make the matrix and then Julia takes over for the actual work
@pyimport numpy as np
Apy = np.diag([1, 2, 3, 4])  # Note that this is Julia array passed to Numpy which comes back as a Julia matrix
eigvals(Apy)

In [ ]:
# Julia has native packages for plotting, but why give up matplotlib
@pyimport matplotlib.pyplot as plt

x = linspace(-1, 1, 100)
y = sin(2π*x)
plt.figure()
plt.plot(x, y)
plt.xlabel("\$x\$")  # Note that dollars have to be escaped for they have special role in Julia's string (interpolation)
plt.show()

Note that you can have a jupyter notebook running Python kernel, import Julia as a module and let the two languages play along nicely.

## WHEN IT _IS NOT_ PYTHON / REASONS FOR SPEED

I do not know/understand the whole story. It seems magical. That said, let me quote the Julia manual:

> Clever application of sufficiently advanced technology can be indistiguishable from magic (A.C. Clarke)

In [ ]:
#Your code is parsed, type annotated(types can be inferred). 
simplef(x, y) = x+y
@code_warntype simplef(1, 2)   # Note x, y have types here

In [ ]:
# When f is called with specific values JIT compiles the function. With type annotation the compiler can reason about
# the code and generate efficient code.
@code_llvm simplef(1, 2)        # Note @julia_simplef_FOO(i64, i64)

In [ ]:
# If called with values of same type. We get a cached function
@code_llvm simplef(1, 2)        # Note @julia_simplef_FOO(i64, i64)

In [ ]:
# Different value types will invoke JIT again
@code_llvm simplef(1, 2.)        # @julia_simplef_BAR(i64, double)

## CALLING C

## METAPROGRAMMING

## BATTERIES INCLUDED

+ Julia has very large scientific stack, e.g. FFT (FFTW by Steven G. Johnson), LA(LAPACK, BLAS, SuiteSparse), statistics
+ Native plotting, e.g. _Gadfly_ package
+ You can use Python modules(`PyCall`), existing C/Fortran libraries(`ccall`)
+ Opportunity to write missing packages!

In [ ]:
@pyimport scipy.fftpack as pyfft
# Show that FFT works
v = rand(20)

fft_jl = FFTW.r2r(v, [FFTW.REDFT10])
fft_py = pyfft.dct(v)

norm(fft_jl - fft_py)

In [ ]:
# Native plotting with Gadfly
using Gadfly
plot(y=fft_jl)

In [11]:
# Linear algebra. The favourite Poisson problem on UnitInterval. Convergence
A = 0  # Keep A outside for loop
for ncells in [2^i for i in 8:14]
    h = 1/ncells
    x = Float64[h*i for i in 0:ncells]
    F = sin(2π*x)
    U = sin(2π*x)/4/π^2
    
    # Based on type of A specialized solver will be called
    A = SymTridiagonal(fill(2, ncells+1), fill(-1, ncells))/h
    b = h*F

    u = A\b
    e = norm(u - U)
    println(e)
end

# Which factorize was dispatched
@which factorize(A)

0.0

factorize(S::SymTridiagonal{T}) at linalg/ldlt.jl:46

05742739384431261
0.00406087309076182
0.002871496220147093
0.0020304589269507247
0.0014357520673215604
0.0010152301641646005
0.0007178761457711251


In [12]:
# And now the eigenvalues. Convergence
for ncells in [2^i for i in 3:11]
    h = 1/ncells
    x = Float64[h*i for i in 0:ncells]

    A = SymTridiagonal(fill(2, ncells+1), fill(-1, ncells))/h
    M = SymTridiagonal([1.; fill(4h/6, ncells-1); 1.], fill(h/6, ncells))
    eigw, eigv = eig(full(A), full(M))
    lmin = minimum(eigw)
    println("System size $(length(eigw)), lmin=$(abs(lmin-π^2))")
end

System size 9, lmin=5.396945915394443
System size 17, lmin=3.094555556451626
System size 33, lmin=1.4650779232335207
System size 65, lmin=0.6797474241796966
System size 129, lmin=0.32442205306690397
System size 257, lmin=0.1582254302874535
System size 513, lmin=0.07811004502728203
System size 1025, lmin=0.03880408264532953
System size 2049, lmin=0.01933930544917395


## ADDITIONAL RESOURCES

+ Wikibook [__Introducing Julia__](https://en.wikibooks.org/wiki/Introducing_Julia)  
+ [__Julia manual__](http://docs.julialang.org/en/release-0.4/) from the homepage
+ YouTube videos
    - Introduction [Steven Johnson](https://www.youtube.com/watch?v=jhlVHoeB05A) @ EuroScipy
    - Introduction [Alan Edelman](https://www.youtube.com/watch?v=37L1OMk_3FU) @ MIT
    - Some metaprogramming [Jeff Bezanson, Stefan Karpinski](https://www.youtube.com/watch?v=pZTqMSM2ksY) @ Curry On
    - Julia + Python [Stefan Karpinksi](https://www.youtube.com/watch?v=PsjANO10KgM)@ PyDATA
    - ... and plenty others from [JuliaCon](http://juliacon.org/). See Julia's [YouTube](https://www.youtube.com/user/JuliaLanguage/videos) channel.
+ Blog posts (mostly metaprogramming)
    - [John Myles White](http://www.johnmyleswhite.com/notebook/category/julia/)
    - [Gray Calhoun](http://gray.clhn.org/blog/julia-macros/)
+ Source code on [GitHub](https://github.com/JuliaLang/julia)